<a href="https://colab.research.google.com/github/sggyuan/GNN/blob/main/Final%20version%E6%9C%80%E5%A5%BD%E8%A1%A8%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import GAE

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = torch.load('/content/drive/MyDrive/my_graph_data-bal.pt')

<ipython-input-4-e9945f3ad513>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('/content/drive/MyDrive/my_graph_data-bal.pt')


In [5]:
data = train_test_split_edges(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
data

Data(x=[16283, 13], y=[100000], val_pos_edge_index=[2, 5000], val_pos_edge_attr=[5000, 8], test_pos_edge_index=[2, 10000], test_pos_edge_attr=[10000, 8], train_pos_edge_index=[2, 99142], train_pos_edge_attr=[99142, 8], train_neg_adj_mask=[16283, 16283], val_neg_edge_index=[2, 5000], test_neg_edge_index=[2, 10000])

In [9]:
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)

        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        # First GCN layer
        x1 = F.leaky_relu(self.bn1(self.conv1(x, edge_index)))
        x1 = self.dropout(x1)

        # Second GCN layer with residual connection
        x2 = F.leaky_relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x2 = self.dropout(x2)

        # Final GCN layer
        x3 = self.conv3(x2, edge_index)

        return x3


In [10]:
out_channels = 2
num_features = data.num_features
epochs = 100

model = GAE(GCNEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.5)

In [11]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [12]:
import torch.nn.functional as F

In [13]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.3846, AP: 0.5192
Epoch: 002, AUC: 0.3268, AP: 0.4989
Epoch: 003, AUC: 0.2192, AP: 0.4221
Epoch: 004, AUC: 0.1351, AP: 0.3568
Epoch: 005, AUC: 0.1015, AP: 0.3342
Epoch: 006, AUC: 0.0896, AP: 0.3245
Epoch: 007, AUC: 0.0926, AP: 0.3215
Epoch: 008, AUC: 0.1383, AP: 0.3359
Epoch: 009, AUC: 0.2691, AP: 0.3910
Epoch: 010, AUC: 0.9011, AP: 0.9010
Epoch: 011, AUC: 0.9230, AP: 0.9297
Epoch: 012, AUC: 0.9295, AP: 0.9373
Epoch: 013, AUC: 0.9308, AP: 0.9392
Epoch: 014, AUC: 0.9313, AP: 0.9400
Epoch: 015, AUC: 0.9309, AP: 0.9402
Epoch: 016, AUC: 0.9304, AP: 0.9401
Epoch: 017, AUC: 0.9299, AP: 0.9400
Epoch: 018, AUC: 0.9292, AP: 0.9396
Epoch: 019, AUC: 0.9277, AP: 0.9389
Epoch: 020, AUC: 0.9260, AP: 0.9380
Epoch: 021, AUC: 0.9252, AP: 0.9376
Epoch: 022, AUC: 0.9241, AP: 0.9371
Epoch: 023, AUC: 0.9227, AP: 0.9364
Epoch: 024, AUC: 0.9216, AP: 0.9358
Epoch: 025, AUC: 0.9214, AP: 0.9357
Epoch: 026, AUC: 0.9212, AP: 0.9356
Epoch: 027, AUC: 0.9205, AP: 0.9353
Epoch: 028, AUC: 0.9220, AP:

In [14]:
from torch_geometric.nn import VGAE

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class VariationalGCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, out_channels)
        self.conv_logstd = GCNConv(hidden_channels, out_channels)

        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x, edge_index):
        # First GCN layer
        x1 = F.leaky_relu(self.bn1(self.conv1(x, edge_index)))
        x1 = self.dropout(x1)

        # Second GCN layer with residual connection
        x2 = F.leaky_relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x2 = self.dropout(x2)

        # Final GCN layers for mean and log-standard deviation
        mu = self.conv_mu(x2, edge_index)
        logstd = self.conv_logstd(x2, edge_index)

        return mu, logstd

    def reparameterize(self, mu, logstd):
        if self.training:
            return mu + torch.randn_like(logstd) * torch.exp(logstd)
        else:
            return mu

In [53]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.5)

In [54]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)

    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [55]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))


Epoch: 001, AUC: 0.6444, AP: 0.6810
Epoch: 002, AUC: 0.6402, AP: 0.6801
Epoch: 003, AUC: 0.6406, AP: 0.6829
Epoch: 004, AUC: 0.6374, AP: 0.6804
Epoch: 005, AUC: 0.6319, AP: 0.6770
Epoch: 006, AUC: 0.6294, AP: 0.6751
Epoch: 007, AUC: 0.6374, AP: 0.6836
Epoch: 008, AUC: 0.6541, AP: 0.6992
Epoch: 009, AUC: 0.6653, AP: 0.7077
Epoch: 010, AUC: 0.6700, AP: 0.7118
Epoch: 011, AUC: 0.6529, AP: 0.6923
Epoch: 012, AUC: 0.5834, AP: 0.6271
Epoch: 013, AUC: 0.5713, AP: 0.6282
Epoch: 014, AUC: 0.5799, AP: 0.6386
Epoch: 015, AUC: 0.5807, AP: 0.6415
Epoch: 016, AUC: 0.5741, AP: 0.6390
Epoch: 017, AUC: 0.5700, AP: 0.6368
Epoch: 018, AUC: 0.5756, AP: 0.6384
Epoch: 019, AUC: 0.5846, AP: 0.6428
Epoch: 020, AUC: 0.5850, AP: 0.6407
Epoch: 021, AUC: 0.5734, AP: 0.6320
Epoch: 022, AUC: 0.5548, AP: 0.6212
Epoch: 023, AUC: 0.5451, AP: 0.6169
Epoch: 024, AUC: 0.5446, AP: 0.6185
Epoch: 025, AUC: 0.5468, AP: 0.6213
Epoch: 026, AUC: 0.5500, AP: 0.6233
Epoch: 027, AUC: 0.5525, AP: 0.6245
Epoch: 028, AUC: 0.5576, AP: